#Basketball-reference Data Normalization and Regression

In [1]:
# special IPython command to prepare the notebook for matplotlib
from collections import defaultdict
from pandas.stats.api import ols
from sklearn.linear_model import LinearRegression

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn import preprocessing
#import statsmodels.api as sm
#import statsmodels as sm


### Read Player Seaaon CSV

In [2]:
df_Season_Collect = pd.read_csv('e:\\nba\\Season_Collection.csv', header=0)

df_Season_Collect.info()
#df_Season_Collect.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3440 entries, 0 to 3439
Data columns (total 33 columns):
Season       3440 non-null object
Age          3440 non-null int64
Tm           3440 non-null object
Lg           3440 non-null object
Pos          3440 non-null object
G            3440 non-null int64
MP           3440 non-null int64
PER          3440 non-null float64
TS_Per       3440 non-null float64
three_Per    3440 non-null float64
FT_Per       3440 non-null float64
ORB_Per      3440 non-null float64
DRB_Per      3440 non-null float64
TRB_Per      3440 non-null float64
AST_Per      3440 non-null float64
STL_Per      3440 non-null float64
BLK_Per      3440 non-null float64
TOV_Per      3440 non-null float64
USG_Per      3440 non-null float64
OWS          3440 non-null float64
DWS          3440 non-null float64
WS           3440 non-null float64
WS48         3440 non-null float64
OBPM         3440 non-null float64
DBPM         3440 non-null float64
BPM          3440 non-null f

In [3]:
year_list = df_Season_Collect['Season'].unique()
print year_list
yearStr = year_list[2]

['1995-96' '1996-97' '1997-98' '1998-99' '1999-00' '2000-01' '2001-02'
 '2002-03' '2003-04' '2004-05' '2005-06' '2006-07' '2007-08' '2008-09'
 '2009-10' '2010-11' '2011-12' '2012-13' '2013-14' '2014-15']


In [4]:
year = df_Season_Collect['Season'].map(lambda x: x == yearStr)
df_year = df_Season_Collect[year]
df_year

,Season,Age,Tm,Lg,Pos,G,MP,PER,TS_Per,three_Per,...,OBPM,DBPM,BPM,VORP,TranFlag,Salary,ORtg,DRtg,EFF,GmSc
9,1997-98,21,BOS,NBA,PG,51,1296,15.4,0.530,0.416,...,1.5,-1.9,-0.3,0.5,N,2384640,105,107,10,8.231373
10,1997-98,19,LAL,NBA,SG,79,2056,18.5,0.548,0.241,...,1.9,-1.7,0.2,1.1,N,1167240,110,106,12,10.331646
11,1997-98,23,LAL,NBA,PG,82,1760,12.9,0.533,0.214,...,-0.5,-0.8,-1.3,0.3,N,703200,109,106,8,5.798780
12,1997-98,22,MIL,NBA,SG,82,3287,16.2,0.539,0.280,...,2.6,-0.8,1.8,3.2,N,2052360,106,108,17,13.017073
13,1997-98,21,MIN,NBA,PF,82,3222,20.4,0.527,0.012,...,1.9,2.9,4.8,5.5,N,2109120,109,103,24,16.618293
14,1997-98,23,PHI,NBA,SF,22,748,14.9,0.539,0.163,...,-0.2,-0.3,-0.5,0.3,N,2583360,99,107,13,10.377273
15,1997-98,23,PHO,NBA,PG,76,1664,15.6,0.556,0.334,...,1.8,-1.2,0.5,1.1,N,1053360,111,105,9,7.086842
16,1997-98,19,POR,NBA,PF,60,808,12.7,0.498,0.009,...,-4.2,1.1,-3.0,-0.2,N,950640,96,100,5,3.315000
17,1997-98,21,SAS,NBA,PF,82,3204,22.6,0.577,0.008,...,1.8,3.7,5.5,6.0,N,2967840,108,95,26,17.382927


###Normalization Function

In [5]:
def Normalization(df_data):
    df_norm = ((df_data - df_data.min()) / (df_data.max() - df_data.min()))
    return df_norm

###Column Index

In [8]:
norm_column = ['Age','G','MP','EFF','GmSc','PER','ORtg','DRtg','AST_Per','STL_Per','BLK_Per',
             'TOV_Per','TS_Per','FT_Per','three_Per','ORB_Per','DRB_Per','TRB_Per','USG_Per','Salary']

###Season Data Normalization

In [9]:
df_tmp = df_year[norm_column] 
df_norm = Normalization(df_tmp)
df_norm

,Age,G,MP,EFF,GmSc,PER,ORtg,DRtg,AST_Per,STL_Per,BLK_Per,TOV_Per,TS_Per,FT_Per,three_Per,ORB_Per,DRB_Per,TRB_Per,USG_Per,Salary
9,0.50,0.483333,0.215833,0.238095,0.349474,0.272727,0.600000,0.923077,0.987179,1.000000,0.000000,0.677419,0.405063,0.703390,1.000000,0.104167,0.000000,0.000,0.580153,0.742476
10,0.00,0.950000,0.515163,0.333333,0.498769,0.585859,0.933333,0.846154,0.529915,0.409091,0.235294,0.064516,0.632911,1.000000,0.571078,0.229167,0.084848,0.128,1.000000,0.204907
11,1.00,1.000000,0.398582,0.142857,0.176556,0.020202,0.866667,0.846154,1.000000,0.590909,0.019608,1.000000,0.443038,0.720339,0.504902,0.020833,0.145455,0.088,0.000000,0.000000
12,0.75,1.000000,1.000000,0.571429,0.689659,0.353535,0.666667,1.000000,0.670940,0.409091,0.039216,0.344086,0.518987,0.423729,0.666667,0.250000,0.187879,0.192,0.778626,0.595750
13,0.50,1.000000,0.974399,0.904762,0.945647,0.777778,0.866667,0.615385,0.623932,0.590909,0.627451,0.000000,0.367089,0.310734,0.009804,0.583333,0.751515,0.704,0.641221,0.620814
14,1.00,0.000000,0.000000,0.380952,0.502012,0.222222,0.200000,0.923077,0.547009,0.590909,0.392157,0.709677,0.518987,0.593220,0.379902,0.208333,0.006061,0.056,0.763359,0.830225
15,1.00,0.900000,0.360772,0.190476,0.268116,0.292929,1.000000,0.769231,0.923077,0.500000,0.019608,0.193548,0.734177,0.000000,0.799020,0.000000,0.090909,0.040,0.419847,0.154621
16,0.00,0.633333,0.023631,0.000000,0.000000,0.000000,0.000000,0.384615,0.000000,0.045455,1.000000,0.548387,0.000000,0.672316,0.002451,1.000000,0.575758,0.752,0.312977,0.109262
17,0.50,1.000000,0.967310,1.000000,1.000000,1.000000,0.800000,0.000000,0.431624,0.000000,0.862745,0.419355,1.000000,0.644068,0.000000,0.854167,1.000000,1.000,0.908397,1.000000


In [10]:
df_Season_norm = pd.DataFrame(columns=df_norm.columns)
df_Season_norm

,Age,G,MP,EFF,GmSc,PER,ORtg,DRtg,AST_Per,STL_Per,BLK_Per,TOV_Per,TS_Per,FT_Per,three_Per,ORB_Per,DRB_Per,TRB_Per,USG_Per,Salary


In [11]:
for yearStr in year_list:
    year = df_Season_Collect['Season'].map(lambda x: x == yearStr)
    df_year = df_Season_Collect[year]
    df_data = df_year[norm_column]
    df_norm = Normalization(df_data)    
    df_Season_norm = df_Season_norm.append(df_norm)

In [12]:
df_Season_norm.info()
df_Season_norm.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3440 entries, 0 to 3439
Data columns (total 20 columns):
Age          3440 non-null float64
G            3440 non-null float64
MP           3440 non-null float64
EFF          3438 non-null float64
GmSc         3440 non-null float64
PER          3440 non-null float64
ORtg         3440 non-null float64
DRtg         3440 non-null float64
AST_Per      3440 non-null float64
STL_Per      3440 non-null float64
BLK_Per      3440 non-null float64
TOV_Per      3440 non-null float64
TS_Per       3440 non-null float64
FT_Per       3440 non-null float64
three_Per    3440 non-null float64
ORB_Per      3440 non-null float64
DRB_Per      3440 non-null float64
TRB_Per      3440 non-null float64
USG_Per      3440 non-null float64
Salary       3440 non-null float64
dtypes: float64(20)
memory usage: 564.4 KB


,Age,G,MP,EFF,GmSc,PER,ORtg,DRtg,AST_Per,STL_Per,BLK_Per,TOV_Per,TS_Per,FT_Per,three_Per,ORB_Per,DRB_Per,TRB_Per,USG_Per,Salary
0,0.00,1.000000,0.000000,NaN,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
1,1.00,0.000000,1.000000,NaN,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
2,0.00,0.702703,0.238183,0.166667,0.189245,0.486486,0.272727,0.230769,0.409774,1.000000,0.363636,0.512821,1.000000,0.555133,0.981308,0.276316,0.153333,0.132075,0.894118,0.240741
3,1.00,0.945946,0.170975,0.000000,0.017191,0.027027,0.000000,0.307692,0.646617,1.000000,0.090909,0.504274,0.000000,0.802281,0.847352,0.039474,0.160000,0.056604,0.200000,0.000000
4,0.75,1.000000,0.765879,0.500000,0.536407,0.513514,1.000000,0.615385,0.413534,0.666667,0.068182,0.025641,0.943396,0.102662,1.000000,0.250000,0.306667,0.235849,0.505882,0.700717
5,0.50,0.864865,0.936854,1.000000,1.000000,1.000000,0.818182,0.230769,0.421053,0.809524,0.931818,0.000000,0.867925,0.000000,0.037383,0.631579,0.693333,0.641509,0.494118,0.629630
6,1.00,0.972973,1.000000,0.611111,0.804154,0.445946,0.363636,1.000000,0.398496,0.619048,0.318182,0.341880,0.622642,1.000000,0.791277,0.315789,0.000000,0.037736,1.000000,1.000000
7,1.00,0.540541,0.083456,0.055556,0.000000,0.000000,0.454545,0.769231,1.000000,0.619048,0.000000,1.000000,0.905660,0.167300,0.956386,0.000000,0.106667,0.000000,0.000000,0.181601
8,0.00,0.000000,0.000000,0.055556,0.030195,0.391892,0.181818,0.000000,0.000000,0.000000,1.000000,0.008547,0.056604,1.000000,0.000000,1.000000,1.000000,1.000000,0.470588,0.128435
9,0.50,0.483333,0.215833,0.238095,0.349474,0.272727,0.600000,0.923077,0.987179,1.000000,0.000000,0.677419,0.405063,0.703390,1.000000,0.104167,0.000000,0.000000,0.580153,0.742476


###Statistical indicators

In [13]:
vars_list = ['Age','G','MP','EFF','GmSc','PER','ORtg','DRtg','AST_Per','STL_Per','BLK_Per',
             'TOV_Per','TS_Per','FT_Per','three_Per','ORB_Per','DRB_Per','TRB_Per','USG_Per']

###Regression

In [14]:
res_Season = ols(y=df_Season_Collect['Salary'], x=df_Season_Collect[vars_list])
res_Season


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Age> + <G> + <MP> + <EFF> + <GmSc> + <PER> + <ORtg> + <DRtg>
             + <AST_Per> + <STL_Per> + <BLK_Per> + <TOV_Per> + <TS_Per> + <FT_Per>
             + <three_Per> + <ORB_Per> + <DRB_Per> + <TRB_Per> + <USG_Per>
             + <intercept>

Number of Observations:         3440
Number of Degrees of Freedom:   20

R-squared:         0.5695
Adj R-squared:     0.5671

Rmse:          3062239.8337

F-stat (19, 3420):   238.1065, p-value:     0.0000

Degrees of Freedom: model 19, resid 3420

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
           Age 455739.5157 14553.0942      31.32     0.0000 427215.4511 484263.5803
             G  4463.0284  5984.6157       0.75     0.4559 -7266.8184 16192.8752

###Normalization Regression

In [15]:
res_Season_norm = ols(y=df_Season_norm['Salary'], x=df_Season_norm[vars_list])
res_Season_norm


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Age> + <G> + <MP> + <EFF> + <GmSc> + <PER> + <ORtg> + <DRtg>
             + <AST_Per> + <STL_Per> + <BLK_Per> + <TOV_Per> + <TS_Per> + <FT_Per>
             + <three_Per> + <ORB_Per> + <DRB_Per> + <TRB_Per> + <USG_Per>
             + <intercept>

Number of Observations:         3438
Number of Degrees of Freedom:   20

R-squared:         0.5651
Adj R-squared:     0.5627

Rmse:              0.1307

F-stat (19, 3418):   233.7530, p-value:     0.0000

Degrees of Freedom: model 19, resid 3418

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
           Age     0.3000     0.0105      28.53     0.0000     0.2794     0.3206
             G    -0.0400     0.0185      -2.17     0.0302    -0.0762    -0.0038
    

###Output Season Normalization Data TO CSV 

In [16]:
def Output_CSV(output_df,location):
    output_file = open(location, "ab")
    output_df.to_csv(output_file,index=False)
    output_file.close()
    
Season_Normalization_location = "e:\\nba\\Season_Normalization.csv"
Output_CSV(df_Season_norm,Season_Normalization_location)

###Read Player Playoffs CSV

In [17]:
df_Playoffs_Collect = pd.read_csv('e:\\nba\\Playoffs_Collection.csv', header=0)

df_Playoffs_Collect.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1595 entries, 0 to 1594
Data columns (total 33 columns):
Season       1595 non-null object
Age          1595 non-null int64
Tm           1595 non-null object
Lg           1595 non-null object
Pos          1595 non-null object
G            1595 non-null int64
MP           1595 non-null int64
PER          1595 non-null float64
TS_Per       1595 non-null float64
three_Per    1595 non-null float64
FT_Per       1595 non-null float64
ORB_Per      1595 non-null float64
DRB_Per      1595 non-null float64
TRB_Per      1595 non-null float64
AST_Per      1595 non-null float64
STL_Per      1595 non-null float64
BLK_Per      1595 non-null float64
TOV_Per      1595 non-null float64
USG_Per      1595 non-null float64
OWS          1595 non-null float64
DWS          1595 non-null float64
WS           1595 non-null float64
WS48         1595 non-null float64
OBPM         1595 non-null float64
DBPM         1595 non-null float64
BPM          1595 non-null f

###Playoffs Data Normalization 

In [18]:
df_Playoffs_norm = pd.DataFrame(columns=df_norm.columns)
df_Playoffs_norm

,Age,G,MP,EFF,GmSc,PER,ORtg,DRtg,AST_Per,STL_Per,BLK_Per,TOV_Per,TS_Per,FT_Per,three_Per,ORB_Per,DRB_Per,TRB_Per,USG_Per,Salary


In [19]:
for yearStr in year_list:
    year = df_Playoffs_Collect['Season'].map(lambda x: x == yearStr)
    df_year = df_Playoffs_Collect[year]
    df_data = df_year[norm_column]
    df_norm = Normalization(df_data)    
    df_Playoffs_norm = df_Playoffs_norm.append(df_norm)

df_Playoffs_norm.info()
df_Playoffs_norm.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1595 entries, 0 to 1594
Data columns (total 20 columns):
Age          1595 non-null float64
G            1595 non-null float64
MP           1595 non-null float64
EFF          1595 non-null float64
GmSc         1595 non-null float64
PER          1595 non-null float64
ORtg         1595 non-null float64
DRtg         1595 non-null float64
AST_Per      1595 non-null float64
STL_Per      1595 non-null float64
BLK_Per      1595 non-null float64
TOV_Per      1595 non-null float64
TS_Per       1595 non-null float64
FT_Per       1595 non-null float64
three_Per    1595 non-null float64
ORB_Per      1595 non-null float64
DRB_Per      1595 non-null float64
TRB_Per      1595 non-null float64
USG_Per      1595 non-null float64
Salary       1595 non-null float64
dtypes: float64(20)
memory usage: 261.7 KB


,Age,G,MP,EFF,GmSc,PER,ORtg,DRtg,AST_Per,STL_Per,BLK_Per,TOV_Per,TS_Per,FT_Per,three_Per,ORB_Per,DRB_Per,TRB_Per,USG_Per,Salary
0,0.0,1.000000,1.000000,0.260870,0.273591,0.810585,0.884956,0.480000,0.534202,0.382353,0.061905,0.518293,1.000000,0.545000,0.829167,0.078947,0.279635,0.109244,0.671429,0.382353
1,1.0,0.571429,0.232558,0.043478,0.032587,0.403900,0.530973,0.480000,1.000000,0.470588,0.000000,1.000000,0.598527,0.273000,0.906250,0.000000,0.328267,0.134454,0.392857,0.000000
2,0.5,0.142857,0.937984,1.000000,1.000000,1.000000,1.000000,1.000000,0.446254,0.500000,0.109524,0.216463,0.915285,0.059000,0.000000,1.000000,0.458967,0.773109,0.000000,1.000000
3,1.0,0.285714,0.085271,0.043478,0.015275,0.320334,0.504425,0.680000,0.381107,1.000000,0.257143,0.554878,0.511971,0.000000,0.883333,0.675439,0.000000,0.000000,0.857143,0.288425
4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.203985
5,0.0,0.833333,0.584906,0.272727,0.346088,0.704280,0.987654,1.000000,0.443223,0.212121,0.048913,0.472477,0.896243,1.000000,0.552553,0.077540,0.378641,0.027027,0.229682,0.204907
6,1.0,1.000000,0.741240,0.318182,0.354114,0.696498,0.987654,0.852941,1.000000,1.000000,0.000000,0.720183,0.864043,0.719595,0.882883,0.000000,0.495146,0.000000,0.000000,0.000000
7,0.5,0.333333,0.514825,0.909091,0.763902,0.809339,0.814815,0.764706,0.747253,0.333333,0.097826,1.000000,0.894454,0.202703,0.000000,0.235294,1.000000,0.635135,0.293286,0.620814
8,1.0,0.250000,0.129380,0.272727,0.280662,0.848249,0.975309,0.794118,0.820513,0.636364,0.000000,0.559633,0.914132,0.750000,0.834835,0.085561,0.941748,0.385135,0.208481,0.154621
9,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.109262


###Regression Playoffs

In [20]:
res_playoffs = ols(y=df_Playoffs_Collect['Salary'], x=df_Playoffs_Collect[vars_list])
res_playoffs


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Age> + <G> + <MP> + <EFF> + <GmSc> + <PER> + <ORtg> + <DRtg>
             + <AST_Per> + <STL_Per> + <BLK_Per> + <TOV_Per> + <TS_Per> + <FT_Per>
             + <three_Per> + <ORB_Per> + <DRB_Per> + <TRB_Per> + <USG_Per>
             + <intercept>

Number of Observations:         1595
Number of Degrees of Freedom:   20

R-squared:         0.5489
Adj R-squared:     0.5435

Rmse:          3514007.4737

F-stat (19, 1575):   100.8708, p-value:     0.0000

Degrees of Freedom: model 19, resid 1575

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
           Age 488565.1787 23059.7632      21.19     0.0000 443368.0427 533762.3146
             G -96760.1699 34974.7574      -2.77     0.0057 -165310.6945 -28209.

###Playoffs Normalization Regression

In [21]:
res_Playoffs_norm = ols(y=df_Playoffs_norm['Salary'], x=df_Playoffs_norm[vars_list])
res_Playoffs_norm


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <Age> + <G> + <MP> + <EFF> + <GmSc> + <PER> + <ORtg> + <DRtg>
             + <AST_Per> + <STL_Per> + <BLK_Per> + <TOV_Per> + <TS_Per> + <FT_Per>
             + <three_Per> + <ORB_Per> + <DRB_Per> + <TRB_Per> + <USG_Per>
             + <intercept>

Number of Observations:         1595
Number of Degrees of Freedom:   20

R-squared:         0.5427
Adj R-squared:     0.5372

Rmse:              0.1666

F-stat (19, 1575):    98.3924, p-value:     0.0000

Degrees of Freedom: model 19, resid 1575

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
           Age     0.3323     0.0182      18.31     0.0000     0.2967     0.3679
             G    -0.1464     0.0332      -4.40     0.0000    -0.2115    -0.0812
    

###Output Playoffs Normalization Data TO CSV¶

In [22]:
Playoffs_Normalization_location = "e:\\nba\\Playoffs_Normalization.csv"
Output_CSV(df_Playoffs_norm,Playoffs_Normalization_location)